In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from liamlib import dotdict
from tensorboardX import SummaryWriter
import os
import pickle
from copy import deepcopy


os.environ['CUDA_VISIBLE_DEVICES'] = '1'
RUN_NAME = 'checkgrad'
writer = SummaryWriter(log_dir=f'./tboard/fc/{RUN_NAME}')
args = dotdict.DotDict()

args.seed = 1
args.batch_size = 64
args.test_batch_size = 100
args.lr = 0.02
args.momentum = 0.9
args.epochs = 10
args.log_interval = 100

use_cuda = not args.no_cuda and torch.cuda.is_available()

torch.manual_seed(args.seed)

device = torch.device("cuda" if use_cuda else "cpu")

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 20, 5, 1)
        self.conv2 = nn.Conv2d(20, 50, 5, 1)
        self.fc1 = nn.Linear(4*4*50, 500)
        self.fc2 = nn.Linear(500, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)
        x = x.view(-1, 4*4*50)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)
    
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(28*28, 1024)
        self.fc2 = nn.Linear(1024, 10)

    def forward(self, x):
        x = x.view(-1, 28*28)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

def train(args, model, device, train_loader, optimizer, epoch, writer):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % args.log_interval == 0:
            prog = 100. * batch_idx / len(train_loader)
            l = loss.item()
            batches = (epoch - 1) * len(train_loader.dataset) + batch_idx * len(data)
            writer.add_scalar('data/train_loss', l, batches)
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset), prog, l))

def test(args, model, device, test_loader, writer, batches):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item() # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True) # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    acc = 100. * correct / len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset), acc))
    writer.add_scalar('data/test_loss', test_loss, batches)
    writer.add_scalar('data/test_acc', acc, batches)
    
def train_checkgrad(args, model, device, train_loader, optimizer, epoch, writer):
    model.train()
    accepts = 0
    accept_or_not_accept = 0
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        half = len(data) // 2
        data0, data1 = data[:half], data[half:]
        target0, target1 = target[:half], target[half:]
        
        with torch.no_grad():
            output1 = model(data1)
            loss_before = F.nll_loss(output1, target1).item()
        
        old_weights = deepcopy(model.state_dict())
        old_opt_state = deepcopy(optimizer.state_dict())
        
        optimizer.zero_grad()
        output0 = model(data0)
        loss = F.nll_loss(output0, target0)
        loss.backward()
        optimizer.step()

        with torch.no_grad():
            output1 = model(data1)
            loss_after = F.nll_loss(output1, target1).item()
        if loss_after >= loss_before:
            model.load_state_dict(old_weights)
            optimizer.load_state_dict(old_opt_state)
        else:
            accepts += 1
        accept_or_not_accept += 1

        if batch_idx % args.log_interval == (args.log_interval - 1):
            prog = 100. * batch_idx / len(train_loader)
            l = loss.item()
            aa = accepts / accept_or_not_accept
            batches = (epoch - 1) * len(train_loader.dataset) + batch_idx * len(data)
            writer.add_scalar('data/train_loss', l, batches)
            writer.add_scalar('data/accept_rate', aa, batches)
            print(aa)
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset), prog, l))
            accepts = 0
            accept_or_not_accept = 0

if 'checkgrad' in RUN_NAME:
    train = train_checkgrad

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=args.batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=args.test_batch_size, shuffle=True, **kwargs)


model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=args.lr, momentum=args.momentum)

for epoch in range(1, args.epochs + 1):
    train(args, model, device, train_loader, optimizer, epoch, writer)
    test(args, model, device, test_loader, writer, epoch * len(train_loader))

if (args.save_model):
    torch.save(model.state_dict(),"mnist_cnn.pt")

0.6
Train Epoch: 1 [6336/60000 (11%)]	Loss: 0.607383
0.39
Train Epoch: 1 [12736/60000 (21%)]	Loss: 0.427610
0.47
Train Epoch: 1 [19136/60000 (32%)]	Loss: 0.113864
0.44
Train Epoch: 1 [25536/60000 (43%)]	Loss: 0.435068
0.44
Train Epoch: 1 [31936/60000 (53%)]	Loss: 0.260550
0.43
Train Epoch: 1 [38336/60000 (64%)]	Loss: 0.283650
0.53
Train Epoch: 1 [44736/60000 (75%)]	Loss: 0.173056
0.41
Train Epoch: 1 [51136/60000 (85%)]	Loss: 0.136700
0.46
Train Epoch: 1 [57536/60000 (96%)]	Loss: 0.077258

Test set: Average loss: 0.1709, Accuracy: 9482/10000 (95%)

0.43
Train Epoch: 2 [6336/60000 (11%)]	Loss: 0.166975
0.51
Train Epoch: 2 [12736/60000 (21%)]	Loss: 0.332341
0.37
Train Epoch: 2 [19136/60000 (32%)]	Loss: 0.116006
0.53
Train Epoch: 2 [25536/60000 (43%)]	Loss: 0.030795
0.46
Train Epoch: 2 [31936/60000 (53%)]	Loss: 0.022938
0.39
Train Epoch: 2 [38336/60000 (64%)]	Loss: 0.121522
0.51
Train Epoch: 2 [44736/60000 (75%)]	Loss: 0.057653
0.42
Train Epoch: 2 [51136/60000 (85%)]	Loss: 0.101950
0.51
Tr